<h1 align=center>Capítulo 5</h1>
<h2 align=center>Arquivos e I/O</h2>
<p align= center><img src=https://t.ctcdn.com.br/RhgceU3jDPpm_52ctxpo1RRZ3AQ=/0x36:799x486/512x288/smart/filters:format(webp)/i333320.jpeg width=500</p>

Todos os programas precisam executar entrada e saída. Este capítulo abrange idiomas comuns para trabalhar com diferentes tipos de arquivos, incluindo arquivos de texto e binários, codificações de arquivos e outros assuntos relacionados. Técnicas para manipular nomes de arquivos e diretórios também são cobertas.

## 5.1. Lendo e escrevendo dados de texto

**Problema**

Você precisa ler ou gravar dados de texto, possivelmente em diferentes codificações de texto, como ASCII, UTF-8 ou UTF-16.

**Solução**

Use a função `open()` com o modo **rt** para ler um arquivo de texto. Por exemplo:

In [1]:
# Read the entire file as a single string
with open('somefile.txt', 'rt', encoding='utf8') as f:
    data = f.read()
    print(data)

In [2]:
# Iterate over the lines of the file
with open('somefile.txt', 'rt', encoding='utf8') as f:
    for line in f:
        # process line
        print(line)


Da mesma forma, para escrever um arquivo de texto, use `open()` com o modo **wt** para escrever um arquivo, limpando e sobrescrevendo o conteúdo anterior (se houver). Por exemplo:

In [3]:
# Write chunks of text data
text1 = 'Esse é um texto para teste.'
text2 = 'Esse é um outro texto para teste'
with open('otherfile.txt', 'wt') as f:
    f.write(text1)
    f.write(text2)

~~~python
# Redirected print statement
with open('otherfile.txt', 'wt') as f:
    print(line1, file=f)
    print(line2, file=f)
    ...
~~~

Para anexar ao final de um arquivo existente, use `open()` com mode **at**. Por padrão, os arquivos são lidos/gravados usando a codificação de texto padrão do sistema, como pode ser encontrado em `sys.getdefaultencoding()`. Na maioria das máquinas, isso é definido como utf-8. Se você sabe que o texto que está lendo ou escrevendo está em uma codificação diferente, forneça o parâmetro de codificação opcional para `open()`. Por exemplo:

~~~python
with open('somefile.txt', 'rt', encoding='latin-1') as f:
    ...
~~~

In [4]:
import sys
sys.getdefaultencoding()

'utf-8'

Python entende várias centenas de codificações de texto possíveis. No entanto, algumas das codificações mais comuns são **ascii, latin-1, utf-8 e utf-16**. O **UTF-8** geralmente é uma aposta segura se estiver trabalhando com aplicativos da web. **ascii** corresponde aos caracteres de 7 bits no intervalo U+0000 a U+007F. **latin-1** é um mapeamento direto de bytes 0-255 para caracteres Unicode U+0000 para U+00FF. A codificação **latin-1** é notável por nunca produzir um erro de decodificação ao ler o texto de uma codificação possivelmente desconhecida. Ler um arquivo como **latin-1** pode não produzir uma decodificação de texto completamente correta, mas ainda pode ser suficiente para extrair dados úteis dele. Além disso, se você gravar os dados posteriormente, os dados de entrada originais serão preservados.

**Discussão**

Ler e escrever arquivos de texto normalmente é muito simples. No entanto, há uma série de aspectos sutis para manter em mente. Primeiro, o uso da instrução `with` nos exemplos estabelece um contexto no qual o arquivo será usado. Quando o controle sai do bloco com, o arquivo será fechado automaticamente. Você não precisa usar a instrução `with`, mas se não usá-la, lembre-se de fechar o arquivo:

In [5]:
f = open('somefile.txt', 'rt')
data = f.read()
f.close()

Outra complicação menor diz respeito ao reconhecimento de novas linhas, que são diferentes no Unix e no Windows (ou seja, **\n** versus **\r\n**). Por padrão, o Python opera no que é conhecido como modo “nova linha universal”. Nesse modo, todas as convenções comuns de nova linha são reconhecidas e os caracteres de nova linha são convertidos em um único caractere **\n** durante a leitura.

Da mesma forma, o caractere de nova linha **\n** é convertido no caractere de nova linha padrão do sistema na saída. Se você não quiser essa tradução, forneça o argumento `newline=''` para `open()`, assim:
~~~python
# Read with disabled newline translation
with open('somefile.txt', 'rt', newline='') as f:
    ...
~~~

Para ilustrar a diferença, aqui está o que você verá em uma máquina Unix se ler o conteúdo de um arquivo de texto codificado pelo Windows contendo os dados brutos `hello world!\r\n`:

In [6]:
# Newline translation enabled (the default)
f = open('hello.txt', 'rt', encoding='utf8')
f.read()

'Olá Mundo!!\n'

In [7]:
# Newline translation disabled
g = open('hello.txt', 'rt', newline='', encoding='utf8')
g.read()

'Olá Mundo!!\r\n'

Uma questão final diz respeito a possíveis erros de codificação em arquivos de texto. Ao ler ou gravar um arquivo de texto, você pode encontrar um erro de codificação ou decodificação. Por exemplo:
~~~python
f = open('sample.txt', 'rt', encoding='ascii')
f.read()
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File "/usr/local/lib/python3.3/encodings/ascii.py", line 26, in decode
        return codecs.ascii_decode(input, self.errors)[0]
 UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position
12: ordinal not in range(128)
>>>
~~~

Se você receber esse erro, geralmente significa que você não está lendo o arquivo na codificação correta. Você deve ler cuidadosamente a especificação do que quer que esteja lendo e verificar se está fazendo certo (por exemplo, lendo dados como UTF-8 em vez de Latin-1 ou o que for necessário). Se erros de codificação ainda são uma possibilidade, você pode fornecer um argumento opcional errors para open() para lidar com os erros. Aqui estão alguns exemplos de esquemas comuns de tratamento de erros:

In [8]:
>>> # Replace bad chars with Unicode U+fffd replacement char
>>> f = open('sample.txt', 'rt', encoding='ascii', errors='replace')
>>> f.read()

''

In [9]:
>>> # Ignore bad chars entirely
>>> g = open('sample.txt', 'rt', encoding='ascii', errors='ignore')
>>> g.read()

''

Se você está constantemente mexendo nos argumentos de codificação e erros para `open()` e fazendo muitos hacks, provavelmente está tornando a vida mais difícil do que precisa. A regra número um com o texto é que você simplesmente precisa ter certeza de que está sempre usando a codificação de texto adequada. Em caso de dúvida, use a configuração padrão (normalmente UTF-8).

## 5.2. Imprimindo em um arquivo

**Problema**

Você deseja redirecionar a saída da função `print()` para um arquivo.

**Solução**

Use o argumento da palavra-chave do arquivo para `print()`, assim:

In [10]:
with open('otherfile.txt', 'r+') as f:
    print('Hello World!', file=f)

**Discussão**

Não há muito mais para imprimir em um arquivo além disso. No entanto, certifique-se de que o arquivo seja aberto no modo de texto. A impressão falhará se o arquivo subjacente estiver no modo binário.

## 5.3. Imprimindo com um separador ou final de linha diferente

**Problema**

Você deseja gerar dados usando `print()`, mas também deseja alterar o caractere separador ou o final da linha.

**Solução**

Use os argumentos das palavras-chave **sep** e **end** para `print()` para alterar a saída como desejar. Por exemplo:

In [11]:
print('ACME', 50, 91.5)

ACME 50 91.5


In [12]:
print('ACME', 50, 91.5, sep=',')

ACME,50,91.5


In [13]:
print('ACME', 50, 91.5, sep=',', end='!!\n')

ACME,50,91.5!!


O uso do argumento final também é como você suprime a saída de novas linhas na saída. Por exemplo:

In [14]:
for i in range(5):
    print(i)

0
1
2
3
4


In [15]:
for i in range(5):
    print(i, end=' ')

0 1 2 3 4 

**Discussão**

Usar `print()` com um separador de item diferente geralmente é a maneira mais fácil de gerar dados quando você precisa de algo diferente de um espaço separando os itens. Às vezes você verá programadores usando `str.join()` para realizar a mesma coisa. Por exemplo:

In [16]:
print(', '.join(['ACME', '50','91.5']))

ACME, 50, 91.5


O problema com `str.join()` é que ele só funciona com strings. Isso significa que muitas vezes é necessário realizar várias acrobacias para que funcione. Por exemplo:
~~~python
row = ('ACME', 50, 91.5)
print(','.join(row))
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
 TypeError: sequence item 1: expected str instance, int found
~~~

In [17]:
row = ('ACME', 50, 91.5)
print(','.join(str(x) for x in row))

ACME,50,91.5


Em vez de fazer isso, você poderia simplesmente escrever o seguinte:

In [18]:
print(*row, sep=',')

ACME,50,91.5


## 5.4. Lendo e gravando dados binários

**Problema**

Você precisa ler ou gravar dados binários, como os encontrados em imagens, arquivos de som e assim por diante.

**Solução**

Use a função `open()` com o modo **rb** ou **wb** para ler ou gravar dados binários. Por exemplo:

In [19]:
# Read the entire file as a single byte string
with open('somefile.bin', 'rb') as f:
    data = f.read()

In [20]:
# Write binary data to a file
with open('somefile.bin', 'wb') as f:
    f.write(b'Hello World')

Ao ler o binário, é importante enfatizar que todos os dados retornados estarão na forma de strings de bytes, não em strings de texto. Da mesma forma, ao escrever, você deve fornecer dados na forma de objetos que expõem dados como bytes (por exemplo, strings de bytes, objetos bytearray etc.).

**Discussão**

Ao ler dados binários, as sutis diferenças semânticas entre strings de bytes e strings de texto representam uma pegadinha em potencial. Em particular, esteja ciente de que a indexação e a iteração retornam valores de bytes inteiros em vez de strings de bytes. Por exemplo:

In [21]:
# Text string
t = 'Hello World'
t[0]

'H'

In [22]:
for c in t:
    print(c)

H
e
l
l
o
 
W
o
r
l
d


In [23]:
# Byte string
b = b'Hello World'
b[0]

72

In [24]:
for c in b:
    print(c)

72
101
108
108
111
32
87
111
114
108
100


Se você precisar ler ou escrever texto de um arquivo em modo binário, lembre-se de decodificá-lo ou codificá-lo. Por exemplo:

In [25]:
with open('somefile.bin', 'rb') as f:
    data = f.read(16)
    text = data.decode('utf-8')

In [26]:
with open('somefile.bin', 'wb') as f:
    text = 'Hello World'
    f.write(text.encode('utf-8'))

Um aspecto menos conhecido da I/O binária é que objetos como arrays e estruturas C podem ser usados para escrita sem nenhum tipo de conversão intermediária para um objeto bytes. Por exemplo:

In [27]:
import array
nums = array.array('i', [1, 2, 3, 4])
with open('data.bin','wb') as f:
    f.write(nums)

Isso se aplica a qualquer objeto que implemente a chamada “interface de buffer”, que expõe diretamente um buffer de memória subjacente a operações que podem funcionar com ele. Escrever dados binários é uma dessas operações.

Muitos objetos também permitem que dados binários sejam lidos diretamente em sua memória subjacente usando o método `readinto()` de arquivos. Por exemplo:

In [28]:
import array
a = array.array('i', [0, 0, 0, 0, 0, 0, 0, 0])
with open('data.bin', 'rb') as f:
    f.readinto(a)

In [29]:
a

array('i', [1, 2, 3, 4, 0, 0, 0, 0])

No entanto, muito cuidado deve ser tomado ao usar essa técnica, pois geralmente é específica da plataforma e pode depender de coisas como o tamanho da palavra e a ordenação dos bytes (ou seja, big endian versus little endian). Veja a Receita 5.9 para outro exemplo de leitura de dados binários em um buffer mutável.

## 5.5. Gravando em um arquivo que ainda não existe

**Problema**

Você deseja gravar dados em um arquivo, mas somente se ele ainda não existir no sistema de arquivos.

**Solução**

Este problema é facilmente resolvido usando o modo **x** pouco conhecido para `open()` em vez do modo **w** usual. Por exemplo:

In [30]:
with open('somefile', 'wt') as f:
    f.write('Hello\n')

~~~python
with open('somefile', 'xt') as f:
    f.write('Hello\n')
    ...
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
 FileExistsError: [Errno 17] File exists: 'somefile'
~~~
Se o arquivo estiver no modo binário, use o modo **xb** em vez de **xt**.

**Discussão**

Esta receita ilustra uma solução extremamente elegante para um problema que às vezes surge ao escrever arquivos (ou seja, substituir acidentalmente um arquivo existente). Uma solução alternativa é primeiro testar o arquivo assim:

In [31]:
import os
if not os.path.exists('somefile'):
    with open('somefile', 'wt') as f:
        f.write('Hello\n')
else:
    print('Arquivo já existe!')

Arquivo já existe!


Claramente, usar o modo de arquivo **x** é muito mais direto. É importante notar que o modo **x** é uma extensão específica do Python 3 para a função `open()`. Em particular, esse modo não existe nas versões anteriores do Python ou nas bibliotecas C subjacentes usadas na implementação do Python.

## 5.6. Executando operações de I/O em uma string

**Problema**

Você deseja alimentar um texto ou uma string binária no código que foi escrito para operar em objetos semelhantes a arquivos.

**Solução**

Use as classes `io.StringIO()` e `io.BytesIO()` para criar objetos semelhantes a arquivos que operam em dados de string. Por exemplo:

In [32]:
import io
s = io.StringIO()
s.write('Hello World\n')

12

In [33]:
s = io.StringIO()
s.write('Hello World\n')
print('This is a test', file=s)

In [34]:
# Get all of the data written so far
s.getvalue()

'Hello World\nThis is a test\n'

In [35]:
# Wrap a file interface around an existing string
s = io.StringIO('Hello\nWorld\n')
s.read(4)

'Hell'

In [36]:
s.read()

'o\nWorld\n'

A classe `io.StringIO` deve ser usada apenas para texto. Se você estiver operando com dados binários, use a classe `io.BytesIO`. Por exemplo:

In [37]:
s = io.BytesIO()
s.write(b'binary data')
s.getvalue()

b'binary data'

**Discussão**

As classes `StringIO` e `BytesIO` são mais úteis em cenários em que você precisa imitar um arquivo normal por algum motivo. Por exemplo, em testes de unidade, você pode usar `StringIO` para criar um objeto semelhante a um arquivo contendo dados de teste que são alimentados em uma função que, de outra forma, funcionaria com um arquivo normal.

Esteja ciente de que as instâncias StringIO e BytesIO não possuem um descritor de arquivo inteiro adequado. Assim, eles não funcionam com código que requer o uso de um arquivo de nível de sistema real, como um arquivo, pipe ou soquete.

## 5.7. Lendo e gravando arquivos de dados compactados

**Problema**

Você precisa ler ou gravar dados em um arquivo com compactação **gzip** ou **bz2**.

**Solução**

Os módulos `gzip` e `bz2` facilitam o trabalho com esses arquivos. Ambos os módulos fornecem uma implementação alternativa de `open()` que pode ser usada para esta finalidade. Por exemplo, para ler arquivos compactados como texto, faça o seguinte:

In [38]:
# gzip compression
import gzip
with gzip.open('somefile.gz', 'rt') as f:
    text = f.read()

In [39]:
# bz2 compression
import bz2
with bz2.open('somefile.bz2', 'rt') as f:
    text = f.read()

Da mesma forma, para gravar dados compactados, faça o seguinte:

In [40]:
# gzip compression
import gzip
with gzip.open('somefile.gz', 'wt') as f:
    f.write(text)

In [41]:
# bz2 compression
import bz2
with bz2.open('somefile.bz2', 'wt') as f:
    f.write(text)

Conforme mostrado, todas as I/O usarão texto e realizarão codificação/decodificação Unicode. Se você quiser trabalhar com dados binários, use um modo de arquivo de rb ou wb.

**Discussão**

Na maioria das vezes, ler ou gravar dados compactados é simples. No entanto, esteja ciente de que escolher o modo de arquivo correto é extremamente importante. Se você não especificar um modo, o modo padrão será binário, o que interromperá os programas que esperam receber texto. Ambos `gzip.open()` e `bz2.open()` aceitam os mesmos parâmetros que a função `open()` integrada, incluindo codificação, erros, nova linha e assim por diante.

Ao gravar dados compactados, o nível de compactação pode ser opcionalmente especificado usando o argumento de palavra-chave `compresslevel`. Por exemplo:

In [42]:
with gzip.open('somefile.gz', 'wt', compresslevel=5) as f:
    f.write(text)

O nível padrão é **9**, que fornece o nível mais alto de compactação. Níveis mais baixos oferecem melhor desempenho, mas não tanta compressão.

Finalmente, um recurso pouco conhecido de `gzip.open()` e `bz2.open()` é que eles podem ser colocados em cima de um arquivo existente aberto em modo binário. Por exemplo, isso funciona:

In [43]:
import gzip
f = open('somefile.gz', 'rb')
with gzip.open(f, 'rt') as g:
    text = g.read()

Isso permite que os módulos `gzip` e `bz2` funcionem com vários objetos semelhantes a arquivos, como soquetes, pipes e arquivos na memória.

## 5.8. Iterando sobre registros de tamanho fixo

**Problema**

Em vez de iterar sobre um arquivo por linhas, você deseja iterar sobre uma coleção de registros ou blocos de tamanho fixo.

**Solução**

Use a função `iter()` e `functools.partial()` usando este truque legal:

In [44]:
from functools import partial
RECORD_SIZE = 32
with open('somefile.data', 'rb') as f:
    records = iter(partial(f.read, RECORD_SIZE), b'')
    for r in records:
        print(r)

b'Em lingu\xc3\xadstica, a no\xc3\xa7\xc3\xa3o de te'
b'xto \xc3\xa9 ampla e ainda aberta a um'
b'a defini\xc3\xa7\xc3\xa3o mais precisa. Gros'
b'so modo, pode ser entendido como'
b' manifesta\xc3\xa7\xc3\xa3o lingu\xc3\xadstica das'
b' ideias de um autor, que ser\xc3\xa3o '
b'interpretadas pelo leitor de aco'
b'rdo com seus conhecimentos lingu'
b'\xc3\xadsticos e culturais. Seu tamanh'
b'o \xc3\xa9 vari\xc3\xa1vel.\r\n\r\n\xe2\x80\x9cConjunto d'
b'e palavras e frases articuladas,'
b' escritas sobre qualquer suporte'
b'\xe2\x80\x9d.[1]\r\n\r\n\xe2\x80\x9cObra escrita consi'
b'derada na sua reda\xc3\xa7\xc3\xa3o original'
b' e aut\xc3\xaantica (por oposi\xc3\xa7\xc3\xa3o a '
b'sum\xc3\xa1rio, tradu\xc3\xa7\xc3\xa3o, notas, com'
b'ent\xc3\xa1rios, etc.)\xe2\x80\x9d.[2]\r\n\r\n"Um t'
b'exto \xc3\xa9 uma ocorr\xc3\xaancia lingu\xc3\xads'
b'tica, escrita ou falada de qualq'
b'uer extens\xc3\xa3o, dotada de unidade'
b' sociocomunicativa, sem\xc3\xa2ntica e'
b' formal. \xc3\x89 uma unidade de

O objeto de registros neste exemplo é um iterável que produzirá pedaços de tamanho fixo até que o final do arquivo seja alcançado. No entanto, esteja ciente de que o **último item** pode ter menos bytes do que o esperado se o tamanho do arquivo não for um múltiplo exato do tamanho do registro.

**Discussão**

Um recurso pouco conhecido da função `iter()` é que ela pode criar um iterador se você passar um valor callable e um valor sentinela. O iterador resultante simplesmente chama o callable fornecido repetidamente até retornar o sentinela, ponto em que a iteração é interrompida.

Na solução, o `functools.partial` é usado para criar um callable que lê um número fixo de bytes de um arquivo cada vez que é chamado. A sentinela de b'' é o que é retornado quando um arquivo é lido, mas o final do arquivo foi atingido.

Por último, mas não menos importante, a solução mostra o arquivo sendo aberto no modo binário. Para ler registros de tamanho fixo, esse provavelmente seria o caso mais comum. Para arquivos de texto, a leitura linha por linha (o comportamento de iteração padrão) é mais comum.

## 5.9. Lendo dados binários em um buffer mutável

**Problema**

Você deseja ler dados binários diretamente em um buffer mutável sem nenhuma cópia intermediária. Talvez você queira alterar os dados no local e gravá-los de volta em um arquivo.

**Solução**

Para ler dados em uma matriz mutável, use o método `readinto()` de arquivos. Por exemplo:

In [45]:
import os.path
def read_into_buffer(filename): 
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf

Aqui está um exemplo que ilustra o uso:

In [46]:
# Write a sample file
with open('sample.bin', 'wb') as f:
    f.write(b'Hello World')

buf = read_into_buffer('sample.bin')
buf

bytearray(b'Hello World')

In [47]:
buf[0:5] = b'Hallo'
buf

bytearray(b'Hallo World')

In [48]:
with open('newsample.bin', 'wb') as f:
    f.write(buf)

**Discussão**

O método `readinto()` de arquivos pode ser usado para preencher qualquer array pré-alocado com dados. Isso inclui até mesmo arrays criados a partir do módulo `array` ou bibliotecas como `numpy`. Ao contrário do método `read()` normal, `readinto()` preenche o conteúdo de um buffer existente em vez de alocar novos objetos e retorná-los. Assim, você poderá usá-lo para evitar alocações extras de memória. Por exemplo, se você estiver lendo um arquivo binário que consiste em registros de tamanhos iguais, poderá escrever um código como este:

In [49]:
record_size = 32 # Size of each record (adjust value)
buf = bytearray(record_size)
with open('somefile', 'rb') as f:
    while True:
        n = f.readinto(buf)
        if n < record_size:
            break
            # Use the contents of buf
            ...
    

Outro recurso interessante para usar aqui pode ser um `memoryview`, que permite fazer fatias de cópia zero de um buffer existente e até mesmo alterar seu conteúdo. Por exemplo:

In [50]:
buf

bytearray(b'Hello\r\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')

In [51]:
m1 = memoryview(buf)
m2 = m1[-5:]
m2

In [52]:
m2[:] = b'WORLD'
buf

bytearray(b'Hello\r\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00WORLD')

Um cuidado ao usar `f.readinto()` é que você deve sempre verificar seu código de retorno, que é o número de bytes realmente lidos.

Se o número de bytes for menor que o tamanho do buffer fornecido, isso pode indicar dados truncados ou corrompidos (por exemplo, se você esperava que um número exato de bytes fosse lido).

Finalmente, fique atento a outras funções relacionadas “into” em vários módulos de biblioteca (por exemplo, `recv_into()`, `pack_into()`, etc.). Muitas outras partes do Python têm suporte para I/O direta ou acesso a dados que podem ser usados para preencher ou alterar o conteúdo de arrays e buffers.

Consulte a Receita 6.12 para obter um exemplo significativamente mais avançado de interpretação de estruturas binárias e uso de visualizações de memória.

## 5.10. Arquivos binários de mapeamento de memória

**Problema**

Você deseja mapear a memória de um arquivo binário em uma matriz de bytes mutável, possivelmente para acesso aleatório ao seu conteúdo ou para fazer modificações no local.

**Solução**

Use o módulo `mmap` para arquivos de mapa de memória. Aqui está uma função de utilitário que mostra como abrir um arquivo e mapeá-lo na memória de maneira portátil:

In [53]:
import os
import mmap
def memory_map(filename, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename, os.O_RDWR)
    return mmap.mmap(fd, size, access=access)

Para utilizar esta função, você precisaria ter um arquivo já criado e preenchido com dados. Aqui está um exemplo de como você pode inicialmente criar um arquivo e expandi-lo para um tamanho desejado:

In [54]:
size = 1000000
with open('data', 'wb') as f:
    f.seek(size-1)
    f.write(b'\x00')

Agora aqui está um exemplo de mapeamento de memória do conteúdo usando a função `memory_map()`:

In [55]:
m = memory_map('data')
len(m)

1000000

In [56]:
m[0:10]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [57]:
m[0]

0

In [58]:
# Reassign a slice
m[0:11] = b'Hello World'
m.close()

In [59]:
# Verify that changes were made
with open('data', 'rb') as f:
    print(f.read(11))

b'Hello World'


O objeto mmap retornado por `mmap()` também pode ser usado como gerenciador de contexto, caso em que o arquivo subjacente é fechado automaticamente. Por exemplo:

In [60]:
with memory_map('data') as m:
    print(len(m))
    print(m[0:10])

1000000
b'Hello Worl'


In [61]:
m.closed

True

Por padrão, a função `memory_map()` mostrada abre um arquivo para leitura e escrita. Quaisquer modificações feitas nos dados são copiadas de volta para o arquivo original. Se o acesso somente leitura for necessário, forneça `map.ACCESS_READ` para o argumento de acesso. Por exemplo:
~~~python
m = memory_map(filename, mmap.ACCESS_READ)
~~~

Se você pretende modificar os dados localmente, mas não deseja que essas alterações sejam gravadas no arquivo original, use `mmap.ACCESS_COPY`:
~~~python
m = memory_map(filename, mmap.ACCESS_COPY)
~~~

**Discussão**

Usar `mmap` para mapear arquivos na memória pode ser um meio eficiente e elegante de acessar aleatoriamente o conteúdo de um arquivo. Por exemplo, em vez de abrir um arquivo e executar várias combinações de chamadas `seek()`, `read()` e `write()`, você pode simplesmente mapear o arquivo e acessar os dados usando operações de fatiamento. Normalmente, a memória exposta por `mmap()` se parece com um objeto `bytearray`. No entanto, você pode interpretar os dados de maneira diferente usando uma visualização de memória. Por exemplo:

In [62]:
m = memory_map('data')
# Memoryview of unsigned integers
v = memoryview(m).cast('I')
v[0] = 7
m[0:4]

b'\x07\x00\x00\x00'

In [63]:
m[0:4] = b'\x07\x01\x00\x00'
v[0]

263

Deve-se enfatizar que o mapeamento de memória de um arquivo não faz com que o arquivo inteiro seja lido na memória. Ou seja, não é copiado em algum tipo de buffer de memória ou array.

Em vez disso, o sistema operacional apenas reserva uma seção de memória virtual para o conteúdo do arquivo. Conforme você acessa diferentes regiões, essas partes do arquivo serão lidas e mapeadas na região de memória conforme necessário. No entanto, partes do arquivo que nunca são acessadas simplesmente permanecem no disco. Tudo isso acontece de forma transparente, nos bastidores.

Se mais de uma memória do interpretador Python mapear o mesmo arquivo, o objeto `mmap` resultante poderá ser usado para trocar dados entre os interpretadores. Ou seja, todos os intérpretes podem ler/gravar dados simultaneamente, e as alterações feitas nos dados de um intérprete aparecerão automaticamente nos outros. Obviamente, alguns cuidados extras são necessários para sincronizar as coisas, mas esse tipo de abordagem às vezes é usado como uma alternativa à transmissão de dados em mensagens por pipes ou sockets.

Como mostrado, esta receita foi escrita para ser o mais geral possível, funcionando tanto no Unix quanto no Windows. Esteja ciente de que existem algumas diferenças de plataforma em relação ao uso da chamada `mmap()` oculta nos bastidores. Além disso, há opções para criar regiões de memória mapeadas anonimamente. Se isso for do seu interesse, certifique-se de ler atentamente a documentação do Python sobre o assunto.

## 5.11. Manipulando nomes de caminho

**Problema**

Você precisa manipular os nomes de caminho para encontrar o nome do arquivo base, o nome do diretório, o caminho absoluto e assim por diante.

**Solução**

Para manipular nomes de caminhos, use as funções no módulo `os.path`. Aqui está um exemplo interativo que ilustra alguns recursos principais:

In [64]:
import os
path = r'\Users\dell\OneDrive\Documentos\4. GitHub\Python_Cookbook_Recipes_3rd\somefile.txt'

In [65]:
# Pegando o último componente do caminho
os.path.basename(path)

'somefile.txt'

In [66]:
# Pegando o nome do diretório
os.path.dirname(path)

'\\Users\\dell\\OneDrive\\Documentos\\4. GitHub\\Python_Cookbook_Recipes_3rd'

In [67]:
# Juntando os componentes
os.path.join('tmp','data',os.path.basename(path))

'tmp\\data\\somefile.txt'

In [68]:
# Expand the user's home directory
path = '~/Python_Cookbook_Recipes_3rd/somefile.txt'
os.path.expanduser(path)

'C:\\Users\\dell/Python_Cookbook_Recipes_3rd/somefile.txt'

In [69]:
# Split the file extension
os.path.splitext(path)

('~/Python_Cookbook_Recipes_3rd/somefile', '.txt')

**Discussão**

Para qualquer manipulação de nomes de arquivos, você deve usar o módulo `os.path` em vez de tentar criar seu próprio código usando as operações de string padrão. Em parte, isso é para portabilidade. O módulo `os.path` conhece as diferenças entre Unix e Windows e pode lidar de forma confiável com nomes de arquivos como **Data/data.csv** e **Data\data.csv**. Em segundo lugar, você realmente não deve gastar seu tempo reinventando a roda. Geralmente, é melhor usar a funcionalidade que já foi fornecida para você.

Deve-se notar que o módulo `os.path` possui muitos outros recursos não mostrados nesta receita. Consulte a documentação para obter mais funções relacionadas ao teste de arquivos, links simbólicos e assim por diante.

## 5.12. Testando a existência de um arquivo

**Problema**

Você precisa testar se existe ou não um arquivo ou diretório.

**Solução**

Use o módulo `os.path` para testar a existência de um arquivo ou diretório. Por exemplo:

In [70]:
import os
os.path.exists('somefile.txt')

True

In [71]:
os.path.exists('spam')

False

Você pode realizar mais testes para ver que tipo de arquivo pode ser. Esses testes retornam `False` se o arquivo em questão não existir:

In [72]:
# Is a regular file
os.path.isfile('somefile.txt')

True

In [73]:
# Is a directory
os.path.isdir('somefile.txt')

False

In [74]:
# Is a symbolic link
os.path.islink('/usr/local/bin/python3')

False

In [75]:
# Get the file linked to
os.path.realpath('/usr/local/bin/python3')

'C:\\usr\\local\\bin\\python3'

Se você precisar obter metadados (por exemplo, o tamanho do arquivo ou a data de modificação), isso também está disponível no módulo `os.path`.

In [76]:
os.path.getsize('somefile.txt')

0

In [77]:
os.path.getmtime('somefile.txt')

1667506668.6204474

In [78]:
import time
time.ctime(os.path.getmtime('somefile.txt'))

'Thu Nov  3 16:17:48 2022'

**Discussão**

O teste de arquivos é uma operação direta usando `os.path`. Provavelmente, a única coisa que você deve estar ciente ao escrever scripts é que talvez você precise se preocupar com permissões, especialmente para operações que obtêm metadados. Por exemplo:
~~~python
os.path.getsize('/Users/guido/Desktop/foo.txt')
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File "/usr/local/lib/python3.3/genericpath.py", line 49, in getsize
      return os.stat(filename).st_size
 PermissionError: [Errno 13] Permission denied: '/Users/guido/Desktop/foo.txt'
~~~

## 5.13. Obtendo uma listagem de diretório

**Problema**

Você deseja obter uma lista dos arquivos contidos em um diretório no sistema de arquivos.

**Solução**

Use a função `os.listdir()` para obter uma lista de arquivos em um diretório:

In [79]:
import os
names = os.listdir('somedir')
names

['Arquivo_teste.ipynb',
 'diretorio',
 'somefile',
 'somefile.bin',
 'somefile.txt']

Isso fornecerá a listagem de diretórios brutos, incluindo todos os arquivos, subdiretórios, links simbólicos e assim por diante. Se você precisar filtrar os dados de alguma forma, considere usar uma *list comphrension* combinada com várias funções na biblioteca `os.path`. Por exemplo:

In [80]:
import os.path
# Get all regular files
names = [name for name in os.listdir('somedir') if os.path.isfile(os.path.join('somedir', name))]
names

['Arquivo_teste.ipynb', 'somefile', 'somefile.bin', 'somefile.txt']

In [81]:
# Get all dirs
dirnames = [name for name in os.listdir('somedir') if os.path.isdir(os.path.join('somedir', name))]
dirnames

['diretorio']

Os métodos `startswith()` e `endswith()` de strings também podem ser úteis para filtrar o conteúdo de um diretório. Por exemplo:

In [82]:
pyfiles = [name for name in os.listdir('somedir') if name.endswith('.ipynb')]
pyfiles

['Arquivo_teste.ipynb']

Para correspondência de nome de arquivo, você pode usar os módulos `glob` ou `fnmatch`. Por exemplo:

In [83]:
import glob
pyfiles = glob.glob('somedir/*.ipynb')
pyfiles

['somedir\\Arquivo_teste.ipynb']

In [84]:
from fnmatch import fnmatch
pyfiles = [name for name in os.listdir('somedir') if fnmatch(name, '*.ipynb')]
pyfiles

['Arquivo_teste.ipynb']

**Discussão**

Obter uma listagem de diretório é fácil, mas fornece apenas os nomes das entradas no diretório. Se você deseja obter metadados adicionais, como tamanhos de arquivo, datas de modificação e assim por diante, você precisa usar funções adicionais no módulo `os.path` ou usar a função `os.stat()`. Para coletar os dados. Por exemplo:

In [85]:
# Example of getting a directory listing
import os
import os.path
import glob
pyfiles = glob.glob('*.ipynb')
pyfiles

['Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb',
 'Capítulo 2 - Strings e Texto.ipynb',
 'Capítulo 3 - Números, Datas e Horas.ipynb',
 'Capítulo 5 - Arquivos.ipynb']

In [86]:
# Get file sizes and modification dates
name_sz_date = [(name, os.path.getsize(name), os.path.getmtime(name)) for name in pyfiles]
for name, size, mtime in name_sz_date:
    print(f"Nome: {name} == Tamanho: {size} -  {mtime}")


Nome: Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb == Tamanho: 123351 -  1667214399.0
Nome: Capítulo 2 - Strings e Texto.ipynb == Tamanho: 155085 -  1667214399.0
Nome: Capítulo 3 - Números, Datas e Horas.ipynb == Tamanho: 117282 -  1667214399.0
Nome: Capítulo 5 - Arquivos.ipynb == Tamanho: 81904 -  1667506660.3621068


In [87]:
# Alternative: Get file metadata
file_metadata = [(name, os.stat(name)) for name in pyfiles]
for name, meta in file_metadata:
    print(f"NOME: {name} == TAMANHO: {meta.st_size} = {meta.st_mtime}")

NOME: Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb == TAMANHO: 123351 = 1667214399.0
NOME: Capítulo 2 - Strings e Texto.ipynb == TAMANHO: 155085 = 1667214399.0
NOME: Capítulo 3 - Números, Datas e Horas.ipynb == TAMANHO: 117282 = 1667214399.0
NOME: Capítulo 5 - Arquivos.ipynb == TAMANHO: 81904 = 1667506660.3621068


Por último, mas não menos importante, esteja ciente de que existem problemas sutis que podem surgir no manuseio de nomes de arquivos relacionados a codificações. Normalmente, as entradas retornadas por uma função como `os.list dir()` são decodificadas de acordo com a codificação de nome de arquivo padrão do sistema. No entanto, é possível, sob certas circunstâncias, encontrar nomes de arquivos não decodificáveis. As receitas 5.14 e 5.15 têm mais detalhes sobre como lidar com esses nomes.

## 5.14. Ignorando a codificação de nome de arquivo

**Problema**

Você deseja executar operações de I/O de arquivo usando nomes de arquivo brutos que não foram decodificados ou codificados de acordo com a codificação de nome de arquivo padrão.

**Solução**

Por padrão, todos os nomes de arquivo são codificados e decodificados de acordo com a codificação de texto retornada por `sys.getfilesystemencoding()`. Por exemplo:

In [88]:
import sys
sys.getfilesystemencoding()

'utf-8'

Se você quiser ignorar essa codificação por algum motivo, especifique um nome de arquivo usando uma cadeia de bytes bruta. Por exemplo:

In [89]:
# Write a file using a unicode filename
with open('jalape\xf1o.txt', 'w') as f:
    f.write('Spicy!')

In [90]:
# Directory listing (raw)
os.listdir(b'.') # Note: byte string

[b'.git',
 b'.idea',
 b'.ipynb_checkpoints',
 b'b\xc3\xa4d.txt',
 b'Cap\xc3\xadtulo 1 - Estruturas de Dados e Algoritmos.ipynb',
 b'Cap\xc3\xadtulo 2 - Strings e Texto.ipynb',
 b'Cap\xc3\xadtulo 3 - N\xc3\xbameros, Datas e Horas.ipynb',
 b'Cap\xc3\xadtulo 5 - Arquivos.ipynb',
 b'data',
 b'data.bin',
 b'hello.txt',
 b'jalape\xc3\xb1o.txt',
 b'LICENSE',
 b'newsample.bin',
 b'otherfile.txt',
 b'README.md',
 b'sample.bin',
 b'sample.txt',
 b'somedir',
 b'somefile',
 b'somefile.bin',
 b'somefile.bz2',
 b'somefile.data',
 b'somefile.gz',
 b'somefile.txt']

In [91]:
# Open file with raw filename
with open(b'jalape\xc3\xb1o.txt') as f:
    print(f.read())

Spicy!


Como você pode ver nas duas últimas operações, o tratamento do nome do arquivo muda ligeiramente quando strings de bytes são fornecidas para funções relacionadas a arquivos, como `open()` e `os.listdir()`.

**Discussão**

Em circunstâncias normais, você não precisa se preocupar com a codificação e decodificação de nome de arquivo - as operações normais de nome de arquivo devem funcionar. No entanto, muitos sistemas operacionais podem permitir que um usuário, por acidente ou malícia, crie arquivos com nomes que não estejam de acordo com as regras de codificação esperadas. Esses nomes de arquivos podem quebrar misteriosamente programas Python que funcionam com muitos arquivos.

Ler diretórios e trabalhar com nomes de arquivos como bytes não codificados brutos tem o potencial de evitar tais problemas, embora ao custo de conveniência de programação.

Consulte a Receita 5.15 para obter uma receita sobre como imprimir nomes de arquivos incodificáveis.

## 5.15. Imprimindo nomes de arquivos incorretos

**Problema**

Seu programa recebeu uma listagem de diretórios, mas quando tentou imprimir os nomes dos arquivos, ele travou com uma exceção UnicodeEncodeError e uma mensagem enigmática sobre “substitutos não permitidos”.

**Solução**

Ao imprimir nomes de arquivos de origem desconhecida, use esta convenção para evitar erros:

In [92]:
def bad_filename(filename):
    return repr(filename)[1:-1]

try:
    print('somefile.txt')

except UnicodeEncodeError:
    print(bad_filename(filename))

somefile.txt


**Discussão**

Esta receita é sobre um problema potencialmente raro, mas muito irritante, relacionado a programas que devem manipular o sistema de arquivos. Por padrão, o Python assume que todos os nomes de arquivo são codificados de acordo com a configuração relatada por `sys.getfilesystemencoding()`. No entanto, certos sistemas de arquivos não impõem necessariamente essa restrição de codificação, permitindo assim que os arquivos sejam criados sem a codificação de nome de arquivo adequada. Não é comum, mas sempre existe o perigo de que algum usuário faça algo bobo e crie esse arquivo por acidente (por exemplo, talvez passando um nome de arquivo incorreto para `open()` em algum código com erros).

Ao executar um comando como `os.listdir()`, nomes de arquivos incorretos deixam o Python em um vínculo. Por um lado, não pode simplesmente descartar nomes ruins. Por outro lado, ainda não pode transformar o nome do arquivo em uma string de texto adequada. A solução do Python para esse problema é pegar um valor de byte não decodificável *\xhh* em um nome de arquivo e mapeá-lo em uma chamada “codificação substituta” representada pelo caractere Unicode *\udchh*. Aqui está um exemplo de como uma listagem de diretório ruim pode parecer se contiver um nome de arquivo *bäd.txt*, codificado como *Latin-1* em vez de *UTF-8*:

In [93]:
import os
files = os.listdir('.')
files

['.git',
 '.idea',
 '.ipynb_checkpoints',
 'bäd.txt',
 'Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb',
 'Capítulo 2 - Strings e Texto.ipynb',
 'Capítulo 3 - Números, Datas e Horas.ipynb',
 'Capítulo 5 - Arquivos.ipynb',
 'data',
 'data.bin',
 'hello.txt',
 'jalapeño.txt',
 'LICENSE',
 'newsample.bin',
 'otherfile.txt',
 'README.md',
 'sample.bin',
 'sample.txt',
 'somedir',
 'somefile',
 'somefile.bin',
 'somefile.bz2',
 'somefile.data',
 'somefile.gz',
 'somefile.txt']

Se você tem um código que manipula nomes de arquivos ou mesmo os passa para funções como `open()`, tudo funciona normalmente. É apenas em situações em que você deseja gerar o nome do arquivo que você encontra problemas (por exemplo, imprimi-lo na tela, registrá-lo etc.). Especificamente, se você tentou imprimir a listagem anterior, seu programa irá travar:
~~~python
for name in files:
    print(name)

spam.py
Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
 UnicodeEncodeError: 'utf-8' codec can't encode character '\udce4' in
position 1: surrogates not allowed
~~~

A razão pela qual ele trava é que o caractere **\udce4** é tecnicamente Unicode inválido. Na verdade, é a segunda metade de uma combinação de dois caracteres conhecida como par substituto.

No entanto, como a primeira metade está faltando, é um Unicode inválido. Assim, a única maneira de produzir uma saída bem-sucedida é tomar uma ação corretiva quando um nome de arquivo incorreto for encontrado. Por exemplo, alterar o código para a receita produz o seguinte:

In [94]:
for name in files:
    try:
        print(name)
    except UnicodeEncodeError:
        print(bad_filename(name))

.git
.idea
.ipynb_checkpoints
bäd.txt
Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb
Capítulo 2 - Strings e Texto.ipynb
Capítulo 3 - Números, Datas e Horas.ipynb
Capítulo 5 - Arquivos.ipynb
data
data.bin
hello.txt
jalapeño.txt
LICENSE
newsample.bin
otherfile.txt
README.md
sample.bin
sample.txt
somedir
somefile
somefile.bin
somefile.bz2
somefile.data
somefile.gz
somefile.txt


A escolha do que fazer para a função `bad_filename()` depende muito de você. Outra opção é recodificar o valor de alguma forma, assim:

In [95]:
def bad_filename(filename):
    temp = filename.encode(sys.getfilesystemencoding(), errors='surrogateescape')
    return temp.decode('latin-1')

O uso desta versão produz a seguinte saída:

In [96]:
 for name in files:
        try:
            print(name)
        except UnicodeEncodeError:
            print(bad_filename(name))

.git
.idea
.ipynb_checkpoints
bäd.txt
Capítulo 1 - Estruturas de Dados e Algoritmos.ipynb
Capítulo 2 - Strings e Texto.ipynb
Capítulo 3 - Números, Datas e Horas.ipynb
Capítulo 5 - Arquivos.ipynb
data
data.bin
hello.txt
jalapeño.txt
LICENSE
newsample.bin
otherfile.txt
README.md
sample.bin
sample.txt
somedir
somefile
somefile.bin
somefile.bz2
somefile.data
somefile.gz
somefile.txt


Esta receita provavelmente será ignorada pela maioria dos leitores. No entanto, se você estiver escrevendo scripts de missão crítica que precisam funcionar de maneira confiável com nomes de arquivos e sistema de arquivos, é algo a se pensar. Caso contrário, você pode ser chamado de volta ao escritório no fim de semana para depurar um erro aparentemente inescrutável.

## 5.16. Adicionando ou alterando a codificação de um arquivo já aberto

**Problema**

Você deseja adicionar ou alterar a codificação Unicode de um arquivo já aberto sem fechá-lo primeiro.

**Solução**

Se você deseja adicionar codificação/decodificação Unicode a um objeto de arquivo já existente que é aberto no modo binário, envolva-o com um objeto `io.TextIOWrapper()`. Por exemplo:

In [97]:
import urllib.request
import io
u = urllib.request.urlopen('http://www.python.org')
f = io.TextIOWrapper(u,encoding='utf-8')
text = f.read()

Se você deseja alterar a codificação de um arquivo em modo de texto já aberto, use seu método ´`detach()` para remover a camada de codificação de texto existente antes de substituí-la por uma nova. Aqui está um exemplo de alteração da codificação em `sys.stdout`:

In [98]:
import sys
sys.stdout.encoding

'UTF-8'

~~~python
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='latin-1')
sys.stdout.encoding

'latin-1'
~~~
Fazer isso pode quebrar a saída do seu terminal. Serve apenas para ilustrar.

**Discussão**

O sistema de I/O é construído como uma série de camadas. Você mesmo pode ver as camadas tentando este exemplo simples envolvendo um arquivo de texto:

In [99]:
f = open('sample.txt', 'w')
f

<_io.TextIOWrapper name='sample.txt' mode='w' encoding='cp1252'>

In [100]:
f.buffer

<_io.BufferedWriter name='sample.txt'>

In [101]:
f.buffer.raw

<_io.FileIO name='sample.txt' mode='wb' closefd=True>

Neste exemplo, `io.TextIOWrapper` é uma camada de manipulação de texto que codifica e decodifica Unicode, `io.BufferedWriter` é uma camada de I/O em buffer que manipula dados binários e `io.FileIO` é um arquivo bruto que representa o descritor de arquivo de baixo nível em o sistema operacional.

Adicionar ou alterar a codificação de texto envolve adicionar ou alterar a camada superior `io.TextIOWrapper`. Como regra geral, não é seguro manipular diretamente as diferentes camadas acessando os atributos mostrados. Por exemplo, veja o que acontece se você tentar alterar a codificação usando esta técnica:

In [102]:
f

<_io.TextIOWrapper name='sample.txt' mode='w' encoding='cp1252'>

In [103]:
f = io.TextIOWrapper(f.buffer, encoding='UTF8')
f

<_io.TextIOWrapper name='sample.txt' encoding='UTF8'>

~~~python
f.write('Hello')
Traceback (most recent call last):
     File "<stdin>", line 1, in <module>
 ValueError: I/O operation on closed file.
~~~

Não funciona porque o valor original de **f** foi destruído e fechou o arquivo subjacente no processo.
O método `detach()` desconecta a camada superior de um arquivo e retorna a próxima camada inferior. Depois, a camada superior não será mais utilizável. Por exemplo:

In [104]:
f = open('sample.txt', 'w')
f

<_io.TextIOWrapper name='sample.txt' mode='w' encoding='cp1252'>

In [105]:
b = f.detach()
b

<_io.BufferedWriter name='sample.txt'>

~~~python
f.write('hello')
Traceback (most recent call last):
 File "<stdin>", line 1, in <module>
ValueError: underlying buffer has been detached
>>>
~~~

Uma vez desvinculado, no entanto, você pode adicionar uma nova camada superior ao resultado retornado. Por exemplo:

In [106]:
f = io.TextIOWrapper(b, encoding='latin-1')
f

<_io.TextIOWrapper name='sample.txt' encoding='latin-1'>

Embora a alteração da codificação tenha sido mostrada, também é possível usar essa técnica para alterar a manipulação de linha, a política de erros e outros aspectos da manipulação de arquivos. Por exemplo:
~~~python
>>> sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='ascii',
... errors='xmlcharrefreplace')
>>> print('Jalape\u00f1o')
Jalape&#241;o
>>>
~~~

Observe como o caractere não ASCII ñ foi substituído por \&#241; na saída.

## 5.17. Escrevendo bytes em um arquivo de texto

**Problema**

Você deseja gravar bytes brutos em um arquivo aberto no modo de texto.

**Solução**

Simplesmente escreva os dados de byte nos arquivos subjacentes ao buffer. Por exemplo:

In [107]:
import sys
sys.stdout.write(b'Hello\n')

Hello


Da mesma forma, os dados binários podem ser lidos de um arquivo de texto lendo de seu atributo buffer.

**Discussão**

O sistema de I/O é construído a partir de camadas. Os arquivos de texto são construídos adicionando uma camada de codificação/decodificação Unicode sobre um arquivo de modo binário em buffer. O atributo buffer simplesmente aponta para este arquivo subjacente. Se você acessá-lo, você ignorará a camada de codificação/decodificação de texto.

O exemplo envolvendo `sys.stdout` pode ser visto como um caso especial. Por padrão, `sys.stdout` é sempre aberto no modo de texto. No entanto, se você estiver escrevendo um script que realmente precisa despejar dados binários na saída padrão, você pode usar a técnica mostrada para ignorar a codificação de texto.

## 5.18. Quebrando um descritor de arquivo existente como um objeto de arquivo

**Problema**

Você tem um descritor de arquivo inteiro correspondendo a um canal de I/O já aberto no sistema operacional (por exemplo, arquivo, pipe, soquete, etc.) e deseja envolver um objeto de arquivo Python de nível superior em torno dele.

**Solução**

Um descritor de arquivo é diferente de um arquivo aberto normal, pois é simplesmente um identificador inteiro atribuído pelo sistema operacional para se referir a algum tipo de canal de I/O do sistema. Se você tiver um descritor de arquivo desse tipo, poderá envolver um objeto de arquivo Python em torno dele usando a função `open()`. No entanto, você simplesmente fornece o descritor de arquivo inteiro como o primeiro argumento em vez do nome do arquivo. Por exemplo:

In [108]:
# Open a low-level file descriptor
import os
fd = os.open('somefile.txt', os.O_WRONLY | os.O_CREAT)
fd

10

In [109]:
# Turn into a proper file
f = open(fd, 'wt')
f.write('hello world\n')
f.close()

Quando o objeto de arquivo de alto nível for fechado ou destruído, o descritor de arquivo subjacente também será fechado. Se isso não for desejado, forneça o argumento opcional `closefd=False` para `open()`. Por exemplo:

In [112]:
# Create a file object, but don't close underlying fd when done
fd = os.open('somefile.txt', os.O_WRONLY | os.O_CREAT)
f = open(fd, 'wt', closefd=False)

**Discussão**

Em sistemas Unix, essa técnica de encapsular um descritor de arquivo pode ser um meio conveniente para colocar uma interface semelhante a um arquivo em um canal de I/O existente que foi aberto de uma maneira diferente (por exemplo, pipes, sockets, etc.). Por exemplo, aqui está um exemplo envolvendo soquetes:

In [114]:
from socket import socket, AF_INET, SOCK_STREAM
def echo_client(client_sock, addr):
    print('Got connection from', addr)
    # Make text-mode file wrappers for socket reading/writing
    client_in = open(client_sock.fileno(), 'rt', encoding='latin-1',
                     closefd=False)
    client_out = open(client_sock.fileno(), 'wt', encoding='latin-1',
                      closefd=False)
    # Echo lines back to the client using file I/O
    for line in client_in:
        client_out.write(line)
        client_out.flush()
    client_sock.close()

def echo_server(address):
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(address)
    sock.listen(1)
    while True:
        client, addr = sock.accept()
        echo_client(client, addr)

É importante enfatizar que o exemplo acima serve apenas para ilustrar um recurso da função `open()` integrada e que funciona apenas em sistemas baseados em Unix. Se você está tentando colocar uma interface semelhante a um arquivo em um soquete e precisa que seu código seja multiplataforma, use o método `makefile()` de soquetes. No entanto, se a portabilidade não for uma preocupação, você descobrirá que a solução acima oferece um desempenho muito melhor do que usar `makefile()`.

Você também pode usar isso para fazer um tipo de alias que permite que um arquivo já aberto seja usado de uma maneira um pouco diferente de como foi aberto pela primeira vez. Por exemplo, veja como você pode criar um objeto de arquivo que permite emitir dados binários no `stdout` (que normalmente é aberto no modo de texto).

Embora seja possível envolver um descritor de arquivo existente como um arquivo adequado, esteja ciente de que nem todos os modos de arquivo podem ser suportados e que certos tipos de descritores de arquivo podem ter efeitos colaterais engraçados (especialmente no que diz respeito ao tratamento de erros, condições de fim de arquivo , etc).

O comportamento também pode variar de acordo com o sistema operacional. Em particular, nenhum dos exemplos provavelmente funcionará em sistemas não-Unix. O resultado final é que você precisará testar minuciosamente sua implementação para garantir que ela funcione conforme o esperado.

## 5.19. Criando arquivos e diretórios temporários

**Problema**

Você precisa criar um arquivo ou diretório temporário para uso quando seu programa for executado. Depois, você possivelmente deseja que o arquivo ou diretório seja destruído.

**Solução**

O módulo tempfile possui uma variedade de funções para realizar esta tarefa. Para criar um arquivo temporário sem nome, use `tempfile.TemporaryFile`:

In [119]:
from tempfile import TemporaryFile
with TemporaryFile('w+t') as f:
    # Read/write to the file
    f.write('Hello World\n')
    f.write('Testing\n')
    # Seek back to beginning and read the data
    f.seek(0)
    data = f.read()
    # Temporary file is destroyed

Ou, se preferir, você também pode usar o arquivo assim:
~~~python
f = TemporaryFile('w+t')
# Use the temporary file
...
f.close()
# File is destroyed
~~~

O primeiro argumento para `TemporaryFile()` é o modo de arquivo, que geralmente é **w+t** para texto e **w+b** para binário. Este modo suporta leitura e escrita simultaneamente, o que é útil aqui, pois fechar o arquivo para alterar os modos o destruiria. `Temporary File()` também aceita os mesmos argumentos que a função interna `open()`. Por exemplo:
~~~python
with TemporaryFile('w+t', encoding='utf-8', errors='ignore') as f:
~~~
Na maioria dos sistemas Unix, o arquivo criado por `TemporaryFile()` não tem nome e nem mesmo terá uma entrada de diretório. Se você quiser relaxar essa restrição, use `NamedTemporaryFile()` em vez disso. Por exemplo:

In [121]:
from tempfile import NamedTemporaryFile
with NamedTemporaryFile('w+t') as f:
    print('filename is:', f.name)

filename is: C:\Users\dell\AppData\Local\Temp\tmpua8ev4jt


Aqui, o atributo `f.name` do arquivo aberto contém o nome do arquivo temporário. Isso pode ser útil se precisar ser fornecido a algum outro código que precise abrir o arquivo. Assim como com `TemporaryFile()`, o arquivo resultante é excluído automaticamente quando é fechado. Se você não quiser isso, forneça um argumento de palavra-chave `delete=False`. Por exemplo:

In [123]:
with NamedTemporaryFile('w+t', delete=False) as f:
    print('filename is:', f.name)

filename is: C:\Users\dell\AppData\Local\Temp\tmpoqosnfdy


Para criar um diretório temporário, use `tempfile.TemporaryDirectory()`. Por exemplo:

In [124]:
from tempfile import TemporaryDirectory
with TemporaryDirectory() as dirname:
    print('dirname is:', dirname)
    # Use the directory
    ...
    # Directory and all contents destroyed

dirname is: C:\Users\dell\AppData\Local\Temp\tmpanb39wk9


**Discussão**

As funções `TemporaryFile()`, `NamedTemporaryFile()` e `temporáriaDirectory()` são provavelmente a maneira mais conveniente de trabalhar com arquivos e diretórios temporários, porque elas tratam automaticamente de todas as etapas de criação e limpeza subsequente.

Em um nível inferior, você também pode usar `mkstemp()` e `mkdtemp()` para criar arquivos e diretórios temporários. Por exemplo:

In [125]:
import tempfile
tempfile.mkstemp()

(10, 'C:\\Users\\dell\\AppData\\Local\\Temp\\tmprlggf_dx')

In [126]:
tempfile.mkdtemp()

'C:\\Users\\dell\\AppData\\Local\\Temp\\tmpgfcgmu5b'

No entanto, essas funções realmente não cuidam do gerenciamento adicional. Por exemplo, a função `mkstemp()` simplesmente retorna um descritor de arquivo do sistema operacional bruto e deixa para você transformá-lo em um arquivo adequado. Da mesma forma, cabe a você limpar os arquivos, se desejar. Normalmente, os arquivos temporários são criados no local padrão do sistema, como `/var/tmp` ou similar. Para descobrir a localização real, use a função `tempfile.gettempdir()`. Por exemplo:

In [127]:
tempfile.gettempdir()

'C:\\Users\\dell\\AppData\\Local\\Temp'

Todas as funções relacionadas a arquivos temporários permitem que você substitua esse diretório, bem como as convenções de nomenclatura, usando os argumentos de palavra-chave **prefix, sufixo e dir**. Por exemplo:
~~~python
>>> f = NamedTemporaryFile(prefix='mytemp', suffix='.txt', dir='/tmp')
>>> f.name
'/tmp/mytemp8ee899.txt'
>>>
~~~

Por último, mas não menos importante, na medida do possível, o módulo `tempfile()` cria arquivos temporários da maneira mais segura possível. Isso inclui apenas dar permissão de acesso ao usuário atual e tomar medidas para evitar condições de corrida na criação do arquivo. Esteja ciente de que pode haver diferenças entre as plataformas. Assim, você deve certificar-se de verificar a documentação oficial para os pontos mais delicados.

O nome do dispositivo varia de acordo com o tipo de dispositivo e sistema operacional. Por exemplo, no Windows, você pode usar um dispositivo de **0**, **1** e assim por diante para abrir as portas de comunicação como **“COM0”** e **“COM1”**. Uma vez aberto, você pode ler e gravar dados usando as chamadas `read()`, `readline()` e `write()`. Por exemplo:
~~~python
ser.write(b'G1 X50 Y50\r\n')
resp = ser.readline()
~~~

Na maioria das vezes, a comunicação serial simples deve ser bastante simples deste ponto em diante.

**Discussão**

Embora simples na superfície, a comunicação serial às vezes pode ficar bastante confusa. Uma razão pela qual você deve usar um pacote como `pySerial` é que ele fornece suporte para recursos avançados (por exemplo, tempos limite, fluxo de controle, liberação de buffer, handshake, etc.). Por exemplo, se você deseja habilitar o handshake RTS-CTS, basta fornecer um argumento `rtscts=True` para `Serial()`. A documentação fornecida é excelente, então há pouco benefício em parafraseá-la aqui.

Tenha em mente que todas as I/O envolvendo portas seriais são binárias. Assim, certifique-se de escrever seu código para usar bytes em vez de texto (ou execute a codificação/decodificação de texto adequada conforme necessário). O módulo `struct` também pode ser útil caso você precise criar comandos ou pacotes com código binário.

## 5.21. Serializando objetos Python

**Problema**

Você precisa serializar um objeto Python em um fluxo de bytes para poder fazer coisas como salvá-lo em um arquivo, armazená-lo em um banco de dados ou transmiti-lo por uma conexão de rede.

**Solução**

A abordagem mais comum para serializar dados é usar o módulo `pickle`. Para despejar um objeto em um arquivo, você faz isso:

In [10]:
import pickle

data = ['Brasil', 'Copa do Mundo', 'São Paulo']
f = open('somefile', 'wb')
pickle.dump(data,f)

Para despejar um objeto em uma string, use `pickle.dumps()`:

In [21]:
s = pickle.dumps(data)

Para recriar um objeto de um fluxo de *bytes*, use as funções `pickle.load()` ou `pickle.loads()`. Por exemplo:

In [19]:
# Restore from a file
f = open('somefile', 'rb')
data = pickle.load(f)
data

['Brasil', 'Copa do Mundo', 'São Paulo']

In [24]:
# Restore from a string
data = pickle.loads(s)
data

['Brasil', 'Copa do Mundo', 'São Paulo']

**Discussão**

Para a maioria dos programas, o uso das funções `dump()` e `load()` é tudo o que você precisa para usar o **pickle** com eficiência. Ele simplesmente funciona com a maioria dos tipos de dados Python e instâncias de classes definidas pelo usuário. 

Se você estiver trabalhando com qualquer tipo de biblioteca que permita fazer coisas como salvar/restaurar objetos Python em bancos de dados ou transmitir objetos pela rede, há uma boa chance de que o **pickle** esteja sendo usado.

**pickle** é uma codificação de dados autodescritiva específica do Python. Por autodescritivo, os dados serializados contêm informações relacionadas ao início e fim de cada objeto, bem como informações sobre seu tipo. Assim, você não precisa se preocupar em definir registros - simplesmente funciona. Por exemplo, se estiver trabalhando com vários objetos, você pode fazer isso:

In [25]:
import pickle
f = open('somedata', 'wb')
pickle.dump([1, 2, 3, 4], f)
pickle.dump('hello', f)
pickle.dump({'Apple', 'Pear', 'Banana'}, f)
f.close()
f = open('somedata', 'rb')
pickle.load(f)


[1, 2, 3, 4]

In [26]:
pickle.load(f)

'hello'

In [27]:
pickle.load(f)

{'Apple', 'Banana', 'Pear'}

Você pode selecionar funções, classes e instâncias, mas os dados resultantes codificam apenas referências de nome para os objetos de código associados. Por exemplo:

In [29]:
import math
import pickle
pickle.dumps(math.cos)

b'\x80\x04\x95\x10\x00\x00\x00\x00\x00\x00\x00\x8c\x04math\x94\x8c\x03cos\x94\x93\x94.'

Quando os dados não são selecionados, supõe-se que toda a fonte necessária esteja disponível. Módulos, classes e funções serão importados automaticamente conforme necessário. Para aplicativos em que os dados do Python estão sendo compartilhados entre interpretadores em máquinas diferentes, esse é um possível problema de manutenção, pois todas as máquinas devem ter acesso ao mesmo código-fonte.
> `pickle.load()` nunca deve ser usado em dados não confiáveis. Como efeito colateral do carregamento, o pickle carregará automaticamente os módulos e criará instâncias. No entanto, um malfeitor que sabe como funciona o pickle pode criar dados “malformados” que fazem com que o Python execute comandos arbitrários do sistema. Assim, é essencial que o pickle seja usado apenas internamente com intérpretes que tenham alguma capacidade de autenticar um ao outro.

Certos tipos de objetos não podem ser conservados. Normalmente, são objetos que envolvem algum tipo de estado externo do sistema, como arquivos abertos, conexões de rede abertas, threads, processos, quadros de pilha e assim por diante. As classes definidas pelo usuário às vezes podem contornar essas limitações fornecendo os métodos `__getstate__()` e `__setstate__()`. Se definido, `pickle.dump()` chamará `__getstate__()` para obter um objeto que pode ser conservado. Da mesma forma, `__setstate__()` será invocado em unpicking. Para ilustrar o que é possível, aqui está uma classe que define internamente um thread, mas ainda pode ser decapado/descongelado:

In [31]:
# countdown.py
import time
import threading
class Countdown:
    def __init__(self, n): 
        self.n = n
        self.thr = threading.Thread(target=self.run)
        self.thr.daemon = True
        self.thr.start()
    
    def run(self):
        while self.n > 0:
            print('T-minus', self.n)
            self.n -= 1
            time.sleep(5)
    def __getstate__(self):
        return self.n
    
    def __setstate__(self, n):
        self.__init__(n)

Tente o seguinte experimento envolvendo decapagem:

In [33]:
c = Countdown(30)

T-minus 30
T-minus 29
T-minus 28
T-minus 27
T-minus 26
T-minus 25
T-minus 24
T-minus 23
T-minus 22
T-minus 21
T-minus 20
T-minus 19
T-minus 18
T-minus 17


In [34]:
# After a few moments
f = open('cstate.p', 'wb')
import pickle
pickle.dump(c, f)
f.close()

T-minus 16
T-minus 15
T-minus 14
T-minus 13
T-minus 12
T-minus 11
T-minus 10
T-minus 9


Agora saia do Python e tente isso após a reinicialização:

In [35]:
f = open('cstate.p', 'rb')
pickle.load(f)

T-minus

 16
T-minus 8
T-minus 15
T-minus 7
T-minus 14
T-minus 6
T-minus 13
T-minus 5
T-minus 12
T-minus 4
T-minus 11
T-minus 3
T-minus 10
T-minus 2
T-minus 9
T-minus 1
T-minus 8
T-minus 7
T-minus 6
T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


Você deve ver o fio magicamente ganhar vida novamente, retomando de onde parou quando você o conservou pela primeira vez.

`pickle` não é uma codificação particularmente eficiente para grandes estruturas de dados, como arrays binários criados por bibliotecas como o módulo array ou numpy. Se você estiver movendo grandes quantidades de dados de array, talvez seja melhor simplesmente salvar dados de array em massa em um arquivo ou usar uma codificação mais padronizada, como HDF5 (suportado por bibliotecas de terceiros).

Por causa de sua natureza específica do Python e anexo ao código-fonte, você provavelmente não deve usar `pickle` como formato para armazenamento de longo prazo. Por exemplo, se o código-fonte for alterado, todos os dados armazenados poderão quebrar e se tornar ilegíveis. Francamente, para armazenar dados em bancos de dados e armazenamento de arquivo, provavelmente é melhor usar uma codificação de dados mais padrão, como XML, CSV ou JSON. Essas codificações são mais padronizadas, suportadas por muitos idiomas diferentes e mais provavelmente adaptadas às alterações em seu código-fonte.

Por último, mas não menos importante, esteja ciente de que o `pickles` tem uma enorme variedade de opções e casos de canto complicados. Para os usos mais comuns, você não precisa se preocupar com eles, mas uma olhada na documentação oficial deve ser necessária se você for construir um aplicativo significativo que use `pickle` para serialização.